<a href="https://colab.research.google.com/github/jared-martin/higher-order-python/blob/main/Higher_Order_Python_Chapter_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3. Cachine and Memoization

We saw in Section 1.8 that a natural recursive function can sometimes perform extremely badly. An easy and general solution to many of these performance problems, as well as some that arise in nonrecursive contexts, is caching.

### 3.1 Caching Fixes Recursion

In [15]:
def fib(month: int) -> int:
    if month < 2:
        return 1
    else:
        return fib(month - 1) + fib(month - 2)

# Takes a few seconds
fib(35)

14930352

As we saw in Section 1.8, this function runs slowly for most arguments, because it wastes time recomputing results it has already computed. For example, `fib(20)` needs to compute `fib(19)` and `fib(18)`, but `fib(19)` also computes `fib(18)`, as well as `fib(17)`, which is also computed once by each of the calls to `fib(18)`. This is a common problem with recursive functions, and it is fixed by caching.

### 3.2 Inline Cachine

The most straightforward way to add caching to a function is to give the function a ~~private hash~~ dictionary.

In [16]:
# In the book, cache is declared outside fib but in a "block" next to fib,
# making the variable static but *not* global.
cache = dict()

def fib(month: int) -> int:
    try:
        return cache[month]
    except KeyError:
        if month < 2:
            cache[month] = 1
        else:
            cache[month] = fib(month - 1) + fib(month - 2)
        return cache[month]

# Instantaneous!
fib(35)

14930352

### 3.3 Good Ideas

Caching comes up over and over in real programs. Almost any program will contain functions where caching might yield a performance win. But the best property of caching is that it’s *mechanical*. If you have a function, and you would like to speed it up, you might rewrite the function, or introduce a better data structure, or a more sophisticated algorithm. This might require ingenuity, which is always in short supply. But adding caching is a no-brainer; the caching transformation is always pretty much the same.

### 3.4 Memoization

Adding the caching code to functions is not very much trouble. And as we saw, the changes required are the same for almost any function. Why not, then, get the computer to do it for us?

In [17]:
# In the book, the author imports a module from the Perl standard library, so
# we'll do the Python equivalent here.
from functools import lru_cache


@lru_cache(maxsize=None)
def fib(month: int) -> int:
    if month < 2:
        return 1
    else:
        return fib(month - 1) + fib(month - 2)

fib(35)

14930352

#### 3.5 The Memoize Module

Memoize gets a function name (or reference) as its argument. It manufactures a new function that maintains a cache and looks up its arguments in the cache. If the new function finds the arguments in the cache, it returns the cached value; if not, it calls the original function, saves the return value in the cache, and returns it to the original caller.

Having manufactured this new function, Memoize then installs it into the Perl symbol table in place of the original function so that when you think you’re calling the original function, you actually get the new cache manager function instead.

In [18]:
from collections.abc import Callable


def memoize(func: Callable) -> Callable:
    cache = dict()
    def stub(*args, **kwargs):
        # This function isn't recursive because it doesn't actually call itself
        key = str(args) + str(kwargs)
        try:
            return cache[key]
        except KeyError:
            cache[key] = func(*args, **kwargs)
            return cache[key]
    return stub


# Note that the syntax in the book - fastfib = memoize(fib) - while valid, won't
# work here, since in Python fastfib is memoized but fib isn't, and fib still
# calls its non-memoized self recursively.
@memoize
def fastfib(month: int) -> int:
    if month < 2:
        return 1
    else:
        return fastfib(month - 1) + fastfib(month - 2)

fastfib(35)

14930352

#### 3.5.2 Lexical Closure

Now another point might be worrying you. Since the value of `func` persists as long as the stub does, what happens if we call memoize a second time, while the first stub is still extant? Will the assignment to `func` in the second call clobber the value that the first stub was using?
The answer is no; everything works perfectly. This is because ~~Perl’s anony- mous functions~~ Python's functions have a property called lexical closure. When an ~~anonymous~~ *(in Python, this holds true for named functions as well)* function is created, Perl packages up its pad, including all the bindings that are in scope, and attaches them to the CV. A function packaged up with an environment in this way is called a closure.

In [21]:
def make_counter(n) -> Callable:
    # In Python, closures don't need to be anonymous, and anonymous functions
    # can't contain assignments, so we use a named function called inner.
    def inner():
        nonlocal n  # In Python, this is needed if we're reassigning n
        print(f'n is {n}')
        n += 1
    return inner

x = make_counter(7)
y = make_counter(20)

x(); x(); x()
y(); y(); y()
x()  # n is the same as it was the first 3 times we invoked x

n is 7
n is 8
n is 9
n is 20
n is 21
n is 22
n is 10


In Python, it feels more natural to use an object to keep track of state than a closure, like this:

In [19]:
class Counter:

    def __init__(self, n):
        self.n = n

    def __call__(self):
        print(f'n is {self.n}')
        self.n += 1

x = Counter(7)
y = Counter(20)
x(); x(); x()
y(); y(); y()
x()

n is 7
n is 8
n is 9
n is 20
n is 21
n is 22
n is 10


#### 3.6.2 Functions with Side Effects

Many functions are called not for their return values but for their side effects.  Suppose you have written a program that formats a computer uptime report and delivers the report to the printer to be printed.  Probably the return value is not interesting, and caching it is silly.  Even if the return value is interesting, memoization is still inappropriate.  The function might complete much more quickly after the first run, because of the memoization, but your boss would not be impressed, because it would have returned the old cached return value immediately, without bothering to actually print the report.

#### 3.6.3 Functions That Return References

Functions that return references to values that may be modified by their callers must not be memoized.  To see the potential problem, consider this example:

In [20]:
@lru_cache(maxsize=None)
def iota(n):
    return list(range(1, n + 1))

i10 = iota(10)
j10 = iota(10)
i10.pop()
print(j10)

[1, 2, 3, 4, 5, 6, 7, 8, 9]


The first call to `iota(10)` generates a new, fresh anonymous ~~array~~ list of the numbers from 1 to 10, and returns a reference to this ~~array~~ list. This reference is automatically placed in the cache, and is also stored into `i10`. The second call to `iota(10)` fetches the same reference from the cache and stores it into `j10`. Both `i10` and `j10` now refer to the same array--we say that they are *aliases* for the array.

When we change the value of the array via the `i10` alias, the change affects the value that is stored in `j10`!  This was probably not what the caller was expecting, and it would not have happened if we had not memoized `iota`. Memoization is supposed to be an optimization. This means it is supposed to speed up the program without changing its behavior.